In [1]:
import pandas as pd
pd.options.display.max_rows = 999
import os
import xlrd
import re
path = '../data/combined_excel.xlsx'
df = pd.read_excel(path,index_col=0, engine='openpyxl')
df['50'] = df['50'].astype(str)

# 第一種case

In [2]:
case1 = {}
for k,v in zip(df['50'].index,df['50'].values):
    if "_x000D" in v:
        case1[k] = v.split('_x000D')[0][:]
res1 = pd.DataFrame()
res1['string_X_train'] = df.loc[case1.keys(),'50'].values
res1['Y_label'] = case1.values()
df = res1

def preprocess(x):
    x = str(x)
    x = re.sub('[\u4e00-\u9fa5]', '', x) # 去除中文
    x = re.sub(r'[^\w\s]','',x) # 去除標點符號
    x = x.replace('\n', '').replace('\r', '').replace('\t', '') # 換行符號去除
    return str.strip(x) # 移除左右空白

df['string_X_train'] = df['string_X_train'].apply(preprocess)
df['Y_label'] = df['Y_label'].apply(preprocess)
print(df.shape)
df.head()

(12378, 2)


,string_X_train,Y_label
0,HONGKONG PROANDHAR GROUP COMPANY_x000D_LIM ITED,HONGKONG PROANDHAR GROUP COMPANY
1,TRAFIGURA PTE LTD_x000D_10 COLLYER QUAY_x000D_...,TRAFIGURA PTE LTD
2,KINGFA SCI AND TECH CO LTD_x000D_ADDRESS NO33 ...,KINGFA SCI AND TECH CO LTD
3,LANDMARK INTL CORP_x000D_SUITE 310GRIFFITH COR...,LANDMARK INTL CORP
4,DVA PLASTICS GmbH_x000D_VALENTINSKAMP 70_x000D...,DVA PLASTICS GmbH


# 起始結束標註

In [3]:
def str2index(context,string):
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye

ys_lst = []
ye_lst = []
for i in range(len(df)):
    context= df['string_X_train'].values[i]
    string = df['Y_label'].values[i]
    ys,ye = str2index(context,string)
    ys_lst.append(ys)
    ye_lst.append(ye)

df['string_Y_1'] = ys_lst
df['string_Y_2'] = ye_lst
df.head()

,string_X_train,Y_label,string_Y_1,string_Y_2
0,HONGKONG PROANDHAR GROUP COMPANY_x000D_LIM ITED,HONGKONG PROANDHAR GROUP COMPANY,0,32
1,TRAFIGURA PTE LTD_x000D_10 COLLYER QUAY_x000D_...,TRAFIGURA PTE LTD,0,17
2,KINGFA SCI AND TECH CO LTD_x000D_ADDRESS NO33 ...,KINGFA SCI AND TECH CO LTD,0,26
3,LANDMARK INTL CORP_x000D_SUITE 310GRIFFITH COR...,LANDMARK INTL CORP,0,18
4,DVA PLASTICS GmbH_x000D_VALENTINSKAMP 70_x000D...,DVA PLASTICS GmbH,0,17


In [4]:
print(df.shape)
df.to_csv('../data/preprocess_for_SQUAD_開狀人.csv')

(12378, 4)


In [5]:
df['predict'] = [str.strip(i.split('_x000D')[0]) for i in df['string_X_train'].values]

In [10]:
開狀人寶典 = pd.DataFrame()
assert [ i==j for i,j in zip(df['predict'].values,df['Y_label'].values)]
開狀人寶典['開狀人'] = list(set(df['predict'].values.tolist()))
print(開狀人寶典.shape)
display(開狀人寶典.head())
開狀人寶典.to_csv('../data/寶典/開狀人寶典.csv')

(1441, 1)


,開狀人
0,CHUZHOU HANSHANG ELECTRICAL
1,HOA AN IMPORT EXPORT JOINT STOCK
2,POSCO INTERNATIONAL CORPORATION
3,MECHEMCO INDUSTRIES
4,EASTERN ASIA TECHNOLOGY LIMITED UNIT 906 9F...


In [152]:
for i in range(len(df)):
    if df['Y_label'].values[i] == df['predict'].values[i]:
        df.loc[i,'correct'] = 'yes'
    else:
        df.loc[i,'correct'] = 'no'
df.head()

,string_X_train,Y_label,string_Y_1,string_Y_2,predict,correct
0,HONGKONG PROANDHAR GROUP COMPANY_x000D_LIM ITED,HONGKONG PROANDHAR GROUP COMPANY,0,32,HONGKONG PROANDHAR GROUP COMPANY,yes
1,TRAFIGURA PTE LTD_x000D_10 COLLYER QUAY_x000D_...,TRAFIGURA PTE LTD,0,17,TRAFIGURA PTE LTD,yes
2,KINGFA SCI AND TECH CO LTD_x000D_ADDRESS NO33 ...,KINGFA SCI AND TECH CO LTD,0,26,KINGFA SCI AND TECH CO LTD,yes
3,LANDMARK INTL CORP_x000D_SUITE 310GRIFFITH COR...,LANDMARK INTL CORP,0,18,LANDMARK INTL CORP,yes
4,DVA PLASTICS GmbH_x000D_VALENTINSKAMP 70_x000D...,DVA PLASTICS GmbH,0,17,DVA PLASTICS GmbH,yes


In [153]:
df['correct'].value_counts()

yes    12378
Name: correct, dtype: int64

In [154]:
# 移除 x000D後保存
df['string_X_train'] = df['string_X_train'].apply(lambda x:x.replace('_x000D_',' '))
df.head()

,string_X_train,Y_label,string_Y_1,string_Y_2,predict,correct
0,HONGKONG PROANDHAR GROUP COMPANY LIM ITED,HONGKONG PROANDHAR GROUP COMPANY,0,32,HONGKONG PROANDHAR GROUP COMPANY,yes
1,TRAFIGURA PTE LTD 10 COLLYER QUAY HEX 2900 OCE...,TRAFIGURA PTE LTD,0,17,TRAFIGURA PTE LTD,yes
2,KINGFA SCI AND TECH CO LTD ADDRESS NO33 KEFENG...,KINGFA SCI AND TECH CO LTD,0,26,KINGFA SCI AND TECH CO LTD,yes
3,LANDMARK INTL CORP SUITE 310GRIFFITH CORPORATE...,LANDMARK INTL CORP,0,18,LANDMARK INTL CORP,yes
4,DVA PLASTICS GmbH VALENTINSKAMP 70 HAMBURGGERMANY,DVA PLASTICS GmbH,0,17,DVA PLASTICS GmbH,yes


In [155]:
from termcolor import colored
import numpy as np

# 實際感受一下
def str2index(context,string):
    if type(string) != str:
        print(string)
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye

def color_output(i,text_input,text_output):
    ys,ye = str2index(text_input,text_output)
    left = text_input[:ys]
    mid = text_output
    right = text_input[ye:]
    print(i,left,colored(mid,'red'),right)

for j in range(40):
    i = np.random.randint(low=0,high=len(df))
    x = df['string_X_train'].values[i]
    p = df['Y_label'].values[i]
    color_output(j,x,p)

0  CRYSTIC RESINS INDIA PVT LTD  196 MATHURA ROAD 121006 HARYANA INDIA
1  PRAIRIE COLTD  NO14LANE 151SEC 2FU SING SOUTH RDTAIPEI TAIWAN
2  KCE TECHNOLOGY CO LTD   SEE FULL ADDRESS FIELD 47A ITEM 2
3  DAP CORP  4041 BOCHERI MIYANGMYEON ANSEONGCITY KYUNGGIDOKOREA T0316770005
4  JASCHEM INTERNATIONAL CORPORATION  SEA MEADOW HOUSE BLACKBURNE HIGHWAY ROAD TOWN TORTOLA BRITISH VIRGIN ISLANDS
5  NAB CORPORATION NIHOMBASHI TS BLDG  4F 116 NIHOMBASHI HISAMATSUCHO CHUOKU TOKYO 1030005 JAPAN
6  GUH CIRCUIT INDUSTRY PG SDN BHD  PART OF PLOT 1240 AND 1241 BAYAN LEPAS FREE INDUSTRIAL ZONEPHASE 3 BAYAN LEPAS 11900 PENANG MALAYSIA
7  BRENNTAG SPA  MILANOFIORI STRADA 6 PAL A13 20057 ASSAGO MI ITALY
8  GRAND PETROCHEMICAL CO LTD  BUILDING NO1 PACIFIC PLAZA NO555 JINGJIA ROAD NINGBO ZHEJIANG PRC
9  POSCO INTERNATIONAL CORPORATION  165 CONVENSIADAERO YEONSUGU INCHEON 21998 KOREA
10  DRACO KING INDUSTRIES LIMITED  ROOM AC 23F GOLDEN SUN CENTRE 5967 BONHAM STRAND WEST SHEUNG WAN HONG KONG
11  SUNTA CHEMICAL

In [156]:
df['predict'].apply(lambda x:x.split(' ')[-1]).value_counts().describe()

count     300.000000
mean       41.260000
std       217.306033
min         1.000000
25%         2.000000
50%         5.000000
75%        14.250000
max      3000.000000
Name: predict, dtype: float64

In [157]:
df ['predict(-1)'] = df['predict'].apply(lambda x:x.split(' ')[-1])
df ['predict(-1)'].value_counts().head(100)

LTD                3000
LIMITED            1710
COLTD              1100
INC                 599
CORPORATION         557
CORP                554
CO                  319
BHD                 313
MATERIALS           259
PETROCHEMICAL       230
INTERNATIONAL       194
BLDG                128
TRADING             114
TECH                112
LLP                 110
GROUP                91
COMPANY              90
INDUSTRIES           90
AND                  84
Limited              82
VIETNAM              79
LLC                  64
SPA                  64
TOKQA                61
KONG                 59
MATERIAL             54
CHEMICALS            54
SONS                 50
NV                   48
PLASTICS             44
CHEMICAL             39
INDONESIA            37
PVTLTD               36
PVT                  36
TECHCOLTD            36
ENTERPRISE           34
HONGKONG             33
SAS                  32
PRIVATE              31
BRANCH               30
ABADI                30
FOR             

In [158]:
開狀人尾綴 = pd.DataFrame()
開狀人尾綴['尾綴'] = df ['predict(-1)'].value_counts().index.tolist()[:20]
# VIETMAN, KONG, TQKQA, PLASTICS, SONS比較怪
開狀人尾綴 = 開狀人尾綴.loc[~開狀人尾綴['尾綴'].isin(['AND','VIETMAN','KONG','TQKQA','PLASTICS','SONS'])]
print(開狀人尾綴.shape)
display(開狀人尾綴)
開狀人尾綴.to_csv('../data/寶典/開狀人尾綴.csv')

(19, 1)


,尾綴
0,LTD
1,LIMITED
2,COLTD
3,INC
4,CORPORATION
5,CORP
6,CO
7,BHD
8,MATERIALS
9,PETROCHEMICAL


In [159]:
df = df.loc[df['predict(-1)'].isin(開狀人尾綴['尾綴'].values.tolist()),:]
print(df.shape)
df.head(30)

(9652, 7)


,string_X_train,Y_label,string_Y_1,string_Y_2,predict,correct,predict(-1)
0,HONGKONG PROANDHAR GROUP COMPANY LIM ITED,HONGKONG PROANDHAR GROUP COMPANY,0,32,HONGKONG PROANDHAR GROUP COMPANY,yes,COMPANY
1,TRAFIGURA PTE LTD 10 COLLYER QUAY HEX 2900 OCE...,TRAFIGURA PTE LTD,0,17,TRAFIGURA PTE LTD,yes,LTD
2,KINGFA SCI AND TECH CO LTD ADDRESS NO33 KEFENG...,KINGFA SCI AND TECH CO LTD,0,26,KINGFA SCI AND TECH CO LTD,yes,LTD
3,LANDMARK INTL CORP SUITE 310GRIFFITH CORPORATE...,LANDMARK INTL CORP,0,18,LANDMARK INTL CORP,yes,CORP
5,OCTENE CO LTD OFFSHORE CHAMBERS PO BOX 217 API...,OCTENE CO LTD,0,13,OCTENE CO LTD,yes,LTD
6,SHANGHAI KINGFA SCI AND TECH DVPT COLTD APPLIC...,SHANGHAI KINGFA SCI AND TECH,0,28,SHANGHAI KINGFA SCI AND TECH,yes,TECH
7,SHANGHAI KINGFA SCI AND TECH DVPT COLTD APPLIC...,SHANGHAI KINGFA SCI AND TECH,0,28,SHANGHAI KINGFA SCI AND TECH,yes,TECH
8,ZHEJIANG MINGRI HOLDINGS GROUP CO LTDADD10FZHE...,ZHEJIANG MINGRI HOLDINGS GROUP CO,0,33,ZHEJIANG MINGRI HOLDINGS GROUP CO,yes,CO
9,ZHEJIANG MINGRI HOLDINGS GROUP CO LTDADD10FZHE...,ZHEJIANG MINGRI HOLDINGS GROUP CO,0,33,ZHEJIANG MINGRI HOLDINGS GROUP CO,yes,CO
10,OJUS PETROCHEMICALS LLP C 289 NIRALA NAGAR LUC...,OJUS PETROCHEMICALS LLP,0,23,OJUS PETROCHEMICALS LLP,yes,LLP


In [160]:
# 移除x000D後和保留常見尾綴後保存
df = df.drop(['predict(-1)'],axis=1)
display(df.head())
print(df.shape)
df.to_csv('../data/preprocess_for_SQUAD_開狀人.csv')

,string_X_train,Y_label,string_Y_1,string_Y_2,predict,correct
0,HONGKONG PROANDHAR GROUP COMPANY LIM ITED,HONGKONG PROANDHAR GROUP COMPANY,0,32,HONGKONG PROANDHAR GROUP COMPANY,yes
1,TRAFIGURA PTE LTD 10 COLLYER QUAY HEX 2900 OCE...,TRAFIGURA PTE LTD,0,17,TRAFIGURA PTE LTD,yes
2,KINGFA SCI AND TECH CO LTD ADDRESS NO33 KEFENG...,KINGFA SCI AND TECH CO LTD,0,26,KINGFA SCI AND TECH CO LTD,yes
3,LANDMARK INTL CORP SUITE 310GRIFFITH CORPORATE...,LANDMARK INTL CORP,0,18,LANDMARK INTL CORP,yes
5,OCTENE CO LTD OFFSHORE CHAMBERS PO BOX 217 API...,OCTENE CO LTD,0,13,OCTENE CO LTD,yes


(9652, 6)
